In [140]:
#%matplotlib notebook
import time
import numpy as np
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d as Axes3D
from Rk4 import solver
#constants
G=4*np.pi**2

mass=np.array([1,3.00273e-6,3.694302981402751714e-30])

#initial velocities
vyE=6.27776068
vyM=-mass[1]*vyE/mass[2]

#initial conditions 
x=np.array([0,1,1+2.5695552898e-6])
y=np.array([0,0,0])
z=np.array([0,0,0])
vx=np.array([0,0,0])
vy=np.array([0,vyE,vyM])
vz=np.array([0,0,0])


dt=0.001
#time grid
t=np.arange(0,1.+dt,dt)
#number of points
N=len(t)

m=len(mass)

#saving coordinates and velocity as six consecutive vectors
q=np.ones((m,6*N))

Total_Energy=np.zeros(N)
Total_L=np.zeros(N)
#initial conditions
q[:,:6] = np.array((x,y,z,vx,vy,vz)).transpose()


#equations
def ODE(t,q,mass):
    #number of planets
    n=len(mass)
    f=np.zeros_like(q)
    
    #x equation
    f[:,:3]=q[:,3:]
    
    #v equation
    for i in range(n):
        for j in range(n):
            if i==j:
                pass
            else:
                xij=q[i,:3]-q[j,:3]
                f[i,3:]+=-G*mass[j]*xij/(np.linalg.norm(xij))**3
    return f

#energy
def total_energy(q,mass):
    m=len(mass)
    KE=0.
    PE=0.
    
    for i in range(m):
        KE+=mass[i]*(np.dot(q[i,3:],q[i,3:]))/2
        for j in range(i):
            xij=q[i,:3]-q[j,:3]
            PE+=-G*mass[i]*mass[j]/np.linalg.norm(xij)
        
    
    return KE+PE
def total_l(q,mass):
    m=len(mass)
    L=0.
    
    for i in range(m):
        r, v=q[i,:3],q[i,3:]
        L+=np.linalg.norm(np.cross(r,mass[i]*v))
    return L

#time evolution
for i in range(1,N):
    q[:,6*(i):6*(i+1)]=solver(dt,t,q[:,6*(i-1):6*i],lambda t,x:ODE(t,x,mass))
    
#Energy     
for i in range(N):
    Total_Energy[i]=total_energy(q[:,6*(i):6*(i+1)],mass)
#L   
for i in range(N):
    Total_L[i]=total_l(q[:,6*(i):6*(i+1)],mass)
    
    
#indexes to access the coordinates and velocities
indx_x=[i for i in range(6*N) if i%6==0]
indx_y=[i for i in range(6*N) if i%6==1]
indx_z=[i for i in range(6*N) if i%6==2]
indx_vx=[i for i in range(6*N) if i%6==3]
indx_vy=[i for i in range(6*N) if i%6==4]
indx_vz=[i for i in range(6*N) if i%6==5]


In [141]:
#plt.scatter(q[0,indx_x],q[0,indx_y],label=f"{i}")
plt.scatter(q[1,indx_x],q[1,indx_y],label=f"{i}")
plt.scatter(q[2,indx_x],q[2,indx_y],label=f"{i}")

plt.legend()
plt.grid()
#plt.gca().set_aspect('equal', adjustable='box')
#plt.savefig(f"rk4_{0.9}ecc.jpeg")
plt.show()

In [79]:
#initial conditions 
x=np.array([1,1+2.5695552898e-6])
y=np.array([0,0])
z=np.array([0,0])
vx=np.array([0,0])
vy=np.array([6.27776068,6.27776068])
vz=np.array([0,0])
mass=np.array([3.00273e-6,3.694302981402751714e-30])

q0= np.array((x,y,z,vx,vy,vz)).transpose()
def ecc(q,mass,v,e0=0):
    #set vy velocity
    q0[1,4]=v
    E=total_energy(q,mass)
    L=total_l(q,mass)
    miu=(mass[0]*mass[1])/(mass[0]+mass[1])
    e=np.sqrt(1+(2*E*L**2)/(miu*(G*mass[0]*mass[1])**2))
    return e-e0